In [0]:
%pip install mlxtend
%pip install js2py

  Obtaining dependency information for mlxtend from https://files.pythonhosted.org/packages/4c/43/2fc7f76c8891aef148901f1ba3dee65c1cbac00a85ae5ee0dabc2b861256/mlxtend-0.23.4-py3-none-any.whl.metadata
  Using cached mlxtend-0.23.4-py3-none-any.whl.metadata (7.3 kB)
  Obtaining dependency information for scikit-learn>=1.3.1 from https://files.pythonhosted.org/packages/a8/f3/62fc9a5a659bb58a03cdd7e258956a5824bdc9b4bb3c5d932f55880be569/scikit_learn-1.6.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached scikit_learn-1.6.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Obtaining dependency information for threadpoolctl>=3.1.0 from https://files.pythonhosted.org/packages/32/d5/f9a850d79b0851d1d4ef6456097579a9005b31fea68726a4ae5f2d82ddd9/threadpoolctl-3.6.0-py3-none-any.whl.metadata
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached mlxtend-0.23.4-py3-none-any.whl (1.4 MB)
Using cached scikit_learn-

In [0]:
%restart_python

FRAUD PATTERN ANALYSIS

In [0]:
%python
import json
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth
import warnings
from collections import defaultdict

# Suppress DeprecationWarnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Flatten the dictionary recursively
def flatten(d, parent_key='', sep='.'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten(v, new_key, sep=sep).items())
        elif isinstance(v, list):
            for idx, item in enumerate(v):
                items.extend(flatten(item, f"{new_key}_{idx}", sep=sep).items())
        else:
            items.append((new_key, str(v)))
    return dict(items)

# Load fraud JSON file
with open("/dbfs/FileStore/tables/proactive/cf_transaction_200.json", "r") as f:
    data = json.load(f)

# Extract relevant attributes (amount, merchant_name, location, ip_address) and flatten
flat_data = []
for txn in data:
    txn_flat = flatten(txn)
    relevant_fields_data = {key: txn_flat[key] for key in ['amount', 'merchant_name', 'location', 'ip_address', 'merchant_category', 'transaction_type'] if key in txn_flat}
    flat_data.append(relevant_fields_data)

# Convert the filtered data into a DataFrame and fill NaN values with "null"
df = pd.DataFrame(flat_data).fillna("null")
print(f"Data Shape: {df.shape}")
print(f"Data Sample: {df.head()}")

# Convert each row to a transaction of feature=value
transactions = []
for _, row in df.iterrows():
    txn = [f"{col}={val}" for col, val in row.items()]
    transactions.append(txn)

# Apply Transaction Encoder
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_tf = pd.DataFrame(te_ary, columns=te.columns_)

# Run FP-Growth with a higher support to find meaningful patterns
frequent_itemsets = fpgrowth(df_tf, min_support=0.3, use_colnames=True)
frequent_itemsets = frequent_itemsets.sort_values(by='support', ascending=False)

# Display top 10 frequent itemsets
print(frequent_itemsets.head(10))

def generate_smart_js_rules(frequent_itemsets, min_len=2, max_rules=20):
    js_rules = []
    merchant_categories = set()

    for _, row in frequent_itemsets.iterrows():
        itemsets = row["itemsets"]
        if len(itemsets) < min_len:
            continue

        # Group by feature
        feature_groups = defaultdict(list)
        for item in itemsets:
            key, val = item.split('=')
            feature_groups[key].append(val)

        conditions = []
        for key, values in feature_groups.items():
            if key == 'merchant_category':
                merchant_categories.update(values)
            elif len(values) == 1:
                conditions.append(f"transaction['{key}'] === '{values[0]}'")
            else:
                value_conditions = [f"transaction['{key}'] === '{val}'" for val in values]
                conditions.append("(" + " || ".join(value_conditions) + ")")

        if merchant_categories:
            merchant_condition = " || ".join([f"transaction['merchant_category'] === '{category}'" for category in merchant_categories])
            conditions.append(f"({merchant_condition})")

        rule = "if (" + " && ".join(conditions) + ") { flagAsFraud(); }"
        js_rules.append(rule)

        if len(js_rules) >= max_rules:
            break

    return js_rules

# Convert frequent patterns into JS rules
js_rules = generate_smart_js_rules(frequent_itemsets, min_len=2, max_rules=20)
for rule in js_rules:
    print(rule)

Data Shape: (213, 6)
Data Sample:   amount               merchant_name  ... merchant_category transaction_type
0   8963                     Cox PLC  ...            Retail  Cash Withdrawal
1   7485             George-Sullivan  ...            Retail  Cash Withdrawal
2   3044  Hughes, Whitney and Romero  ...     Entertainment           Refund
3   1792                 Meadows Inc  ...   Online Services  Cash Withdrawal
4   2015               Barnett-Flynn  ...            Retail         Transfer

[5 rows x 6 columns]
    support                                           itemsets
0  1.000000                          (ip_address=172.156.11.1)
3  0.361502                (merchant_category=Online Services)
6  0.361502  (ip_address=172.156.11.1, merchant_category=On...
1  0.323944                         (merchant_category=Retail)
4  0.323944  (ip_address=172.156.11.1, merchant_category=Re...
2  0.314554                  (merchant_category=Entertainment)
5  0.314554  (ip_address=172.156.11.1, me

In [0]:
js_file_path = "/dbfs/FileStore/tables/generated_fraud_rule.js"

with open(js_file_path, "w") as f:
    f.write("//auto generated fraud rules\n")
    for rule in js_rules:
        f.write(rule + "\n")


with open(js_file_path, "r",encoding="utf-8") as f:
    js_content = f.read()

print(js_content)

# DBTITLE 1

//auto generated fraud rules
if (transaction['ip_address'] === '172.156.11.1' && (transaction['merchant_category'] === 'Online Services')) { flagAsFraud(); }
if (transaction['ip_address'] === '172.156.11.1' && (transaction['merchant_category'] === 'Online Services' || transaction['merchant_category'] === 'Retail')) { flagAsFraud(); }
if (transaction['ip_address'] === '172.156.11.1' && (transaction['merchant_category'] === 'Online Services' || transaction['merchant_category'] === 'Entertainment' || transaction['merchant_category'] === 'Retail')) { flagAsFraud(); }



In [0]:
%pip install js2py

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


EXECUTE RULES ON TRANSACTIONS

In [0]:
import js2py
import json


def wrap_js_rule(rule):
    return f"""
    function wrapRule(transaction, flagAsFraud) {{ {rule} }}
    """

def execute_js_rules(transaction1, js_rules):
    results = []

    for rule in js_rules:
        is_fraud = False
        def flagAsFraud():
            nonlocal is_fraud
            is_fraud = True

        try:
            rule1 = wrap_js_rule(rule)
            js_func = js2py.eval_js(rule1)
            js_func(transaction1, flagAsFraud)
        except Exception as e:
            print(f"Error executing rule:\n{rule}\nReason: {e}")
            continue

        if is_fraud:
            results.append(rule)

       

    return results, is_fraud


In [0]:
import json
import os

def append_to_rule_log(transaction_json, rules_fired, is_fraud, log_path="/dbfs/FileStore/tables/proactive/3log.jsonl"):
    log_entry = {
        "transaction_Date": transaction_json.get("transaction_date", "000000"),
        "transaction_id": transaction_json.get("transaction_id", "N/A"),
        "transactionAmount": transaction_json.get("amount", "N/A"),
        "is_fraud": is_fraud,
        "rules_fired": rules_fired
    }

    # Read existing log entries
    logs = []
    if os.path.exists(log_path):
        with open(log_path, "r") as f:
            logs = [json.loads(line) for line in f if line.strip()]

    # Append the new log entry
    logs.append(log_entry)
    
    # Overwrite the file with all entries (including the new one)
    with open(log_path, "w") as f:
        for entry in logs:
            f.write(json.dumps(entry) + "\n")

    print(f"✅ Logged transaction {log_entry['transaction_id']} with {len(rules_fired)} rule(s) fired.")
    print(log_entry)

In [0]:
with open("/dbfs//FileStore/tables/proactive/transactions_500.json", "r") as f:
    transactions = json.load(f)
print(transactions)

print(type(js_rules))

def run_bulk_validation(transactions, js_rules, log_path="/dbfs//FileStore/tables/proactive/3log.jsonl"):
    from datetime import datetime

    # Process transactions
    for txn in transactions:
        rules_fired, is_fraud_rec = execute_js_rules(txn, js_rules)
        print(is_fraud_rec)
       # is_fraud = txn.get("is_fraud", False)  # Assuming this exists in your test JSON
        if is_fraud_rec:
            append_to_rule_log(txn, rules_fired, is_fraud_rec, log_path)

run_bulk_validation(transactions, js_rules)

[{'transaction_id': 'TXN_custom_364', 'customer_id': 'CUST_custom_364', 'transaction_date': '2024-06-03', 'amount': '6973', 'transaction_type': 'Transfer', 'merchant_name': 'York, Washington and Garza', 'merchant_category': 'Automotive', 'location': 'Mcclurestad, Liberia', 'ip_address': '172.156.11.1'}, {'transaction_id': 'TXN_custom_307', 'customer_id': 'CUST_custom_307', 'transaction_date': '2024-11-19', 'amount': '1860', 'transaction_type': 'Bill Payment', 'merchant_name': 'Mcneil Group', 'merchant_category': 'Entertainment', 'location': 'West Matthewmouth, Suriname', 'ip_address': '172.156.11.1'}, {'transaction_id': 'TXN_custom_393', 'customer_id': 'CUST_custom_393', 'transaction_date': '2024-08-22', 'amount': '8963', 'transaction_type': 'Cash Withdrawal', 'merchant_name': 'Cox PLC', 'merchant_category': 'Retail', 'location': 'West Samuelchester, Anguilla', 'ip_address': '172.156.11.1'}, {'transaction_id': 'TXN_custom_124', 'customer_id': 'CUST_custom_124', 'transaction_date': '202

In [0]:
import json

def load_confirmed_fraud_ids_from_json(json_path):
    with open(json_path, "r") as f:
        data = json.load(f)
        # print(set(item['transaction_id'] for item in data))
        return set(item["transaction_id"] for item in data)


def evaluate_rule_validity_against_confirmed_json(log_file_path, confirmed_json_path):
    confirmed_fraud_ids = load_confirmed_fraud_ids_from_json(confirmed_json_path)
    model_fraud_ids=[]
    with open(log_file_path, "r") as f:
        for line in f:
            model_fraud_ids.append(json.loads(line))
  
    total_confirmed = len(confirmed_fraud_ids)
    total_model = len(model_fraud_ids)
    print(total_model)

    rule_true_hits = {}  # rule → count of true confirmed frauds it detected

    with open(log_file_path, "r") as f:
        for line in f:
            record = json.loads(line)
            # print(record)
            transaction_id = record.get("transaction_id")
            is_fraud = record.get("is_fraud", False)
            rules = record.get("rules_fired", [])

            # Only count as true hit if:
            # - is_fraud is True AND
            # - transactionId is in confirmed fraud list
            if not is_fraud or transaction_id not in confirmed_fraud_ids:
                continue

            for rule in rules:
                rule_true_hits[rule] = rule_true_hits.get(rule, 0) + 1

    # Compute rule accuracy
    rule_accuracies = {}
    for rule, count in rule_true_hits.items():
        accuracy = count / total_confirmed if total_confirmed else 0
        rule_accuracies[rule] = round(accuracy, 4)

    return sorted(
        rule_accuracies.items(), key=lambda x: -x[1]
    )  # sorted by accuracy descending

In [0]:
results = evaluate_rule_validity_against_confirmed_json(
    log_file_path="/dbfs/FileStore/tables/proactive/3log.jsonl",
    confirmed_json_path="/dbfs/FileStore/tables/proactive/cf_transaction_200.json"
)

print(len(results)-1)

for rule, accuracy in results:
    print(f"✅ Rule: {rule}\n   Accuracy vs confirmed frauds: {accuracy}")


221
2
✅ Rule: if (transaction['ip_address'] === '172.156.11.1' && (transaction['merchant_category'] === 'Online Services' || transaction['merchant_category'] === 'Entertainment' || transaction['merchant_category'] === 'Retail')) { flagAsFraud(); }
   Accuracy vs confirmed frauds: 1.0
✅ Rule: if (transaction['ip_address'] === '172.156.11.1' && (transaction['merchant_category'] === 'Online Services' || transaction['merchant_category'] === 'Retail')) { flagAsFraud(); }
   Accuracy vs confirmed frauds: 0.6854
✅ Rule: if (transaction['ip_address'] === '172.156.11.1' && (transaction['merchant_category'] === 'Online Services')) { flagAsFraud(); }
   Accuracy vs confirmed frauds: 0.3615
